# Stock Data ETL

#### This notebook uses the Yahoo Finance API to pull historical stock market data for the S&P 500 index. Then, I group by week and write out the data to a csv file which is then stored in a database. 

In [18]:
import pandas as pd
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr

#### The line below downloads data for the S&P 500 (stock ticker SPY) dating from 2020-01-01 to 2023-11-24.

In [46]:
data = yf.download("SPY", start="2020-01-01", end="2023-11-24")

[*********************100%%**********************]  1 of 1 completed


In [47]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,323.540009,324.890015,322.529999,324.869995,306.295258,59151200
2020-01-03,321.160004,323.640015,321.100006,322.410004,303.975922,77709700
2020-01-06,320.489990,323.730011,320.359985,323.640015,305.135651,55653900
2020-01-07,323.019989,323.540009,322.239990,322.730011,304.277649,40496400
2020-01-08,322.940002,325.779999,322.670013,324.450012,305.899292,68296000


In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 981 entries, 2020-01-02 to 2023-11-22
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       981 non-null    float64
 1   High       981 non-null    float64
 2   Low        981 non-null    float64
 3   Close      981 non-null    float64
 4   Adj Close  981 non-null    float64
 5   Volume     981 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 53.6 KB


The index is a datetime index. I will drop the datetime index and instead create a new column called Date.

The data types are float64 because they are prices for the S&P 500. The volume is an int64 as it represents the number of shares traded during that day. 

In [49]:
date = data.index.to_numpy()
data.reset_index(drop=True, inplace=True)
data["Date"] = date

In [50]:
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Date'], dtype='object')

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Open       981 non-null    float64       
 1   High       981 non-null    float64       
 2   Low        981 non-null    float64       
 3   Close      981 non-null    float64       
 4   Adj Close  981 non-null    float64       
 5   Volume     981 non-null    int64         
 6   Date       981 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 53.8 KB


I will now resample the data to disaggregate by week:

In [55]:
weekly_resampled_data = data.set_index("Date").resample("W").ffill()

In [58]:
weekly_resampled_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-05,321.160004,323.640015,321.100006,322.410004,303.975922,77709700
2020-01-12,327.290009,327.459991,325.200012,325.709991,307.087280,53029300
2020-01-19,331.700012,332.179993,330.850006,331.950012,312.970490,95846000
2020-01-26,332.440002,332.529999,327.359985,328.769989,309.972260,87578400
2020-02-02,327.000000,327.170013,320.730011,321.730011,303.334869,113845600


Now, I can save the data to a CSV file:

In [59]:
weekly_resampled_data.to_csv("spy_data", sep=',')